<h1>Convolutional Neural Network on MNIST</h1>

<h2><b>What are we talking about?</b></h2>
<p>Convolutional networks were inspired by biological processes in that the connectivity pattern between neurons resembles the organization of the animal visual cortex. Individual cortical neurons respond to stimuli only in a restricted region of the visual field known as the receptive field. The receptive fields of different neurons partially overlap such that they cover the entire visual field.</p>
<p>They have applications in image and video recognition, recommender systems and natural language processing.</p>
<h2><b>Architecture of a CNN</b></h2>
<img src="img/cnn.png" width="600" height="400"></img>

<h3><i>Convolutional layers</i></h3>
<p>Convolutional layers apply a convolution operation to the input, passing the result to the next layer.
The convolution emulates the response of an individual neuron to visual stimuli.</p>

<h3><i>Pooling layers</i></h3>
<p>They combine the outputs of neuron clusters at one layer into a single neuron in the next layer.</p>
<p>Examples: <b>max pooling</b> and <b>avg pooling</b></p>

<h2>References and more infos: </h2>
<ul>
    <li><a href="https://en.wikipedia.org/wiki/Convolutional_neural_network">Wiki</a></li>
    <li><a href=https://deeplearning4j.org/convolutionalnetwork>More infos</a></li>
</ul>


In [0]:
# Import dependencies
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
height = 28
width = 28
channels = 1
n_inputs = height * width

# Feature maps --> a layer full of neurons using the same filters
# Kernels --> filter's variables
# Stride --> the distance between two consecutive receptive fields
# Padding --> VALID if the convolutional layer does not use zero padding, SAME if uses zero padding if necessary
conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 128

conv2_dropout_rate = 0.25
fc1_dropout_rate = 0.5

n_outputs = 10

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')

# First convolutional layer
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")

# Second convolutional layer
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

# Pooling: subsample the image to reduce the computational load
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])
    pool3_flat_drop = tf.layers.dropout(pool3_flat, conv2_dropout_rate, training=training)

# Fully connected layer
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat_drop, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [0]:
mnist = input_data.read_data_sets("/tmp/data/")

In [0]:
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

In [5]:
n_epochs = 500
batch_size = 50

best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images,
                                                y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                           y: mnist.validation.labels})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                        y: mnist.test.labels})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "./my_mnist_model")

Epoch 0, train accuracy: 100.0000%, valid. accuracy: 98.1600%, valid. best loss: 0.061918
Epoch 1, train accuracy: 100.0000%, valid. accuracy: 98.8400%, valid. best loss: 0.046071
Epoch 2, train accuracy: 100.0000%, valid. accuracy: 98.9800%, valid. best loss: 0.039397
Epoch 3, train accuracy: 100.0000%, valid. accuracy: 99.1600%, valid. best loss: 0.036479
Epoch 4, train accuracy: 100.0000%, valid. accuracy: 99.0000%, valid. best loss: 0.032411
Epoch 5, train accuracy: 100.0000%, valid. accuracy: 99.0600%, valid. best loss: 0.031912
Epoch 6, train accuracy: 100.0000%, valid. accuracy: 99.0400%, valid. best loss: 0.031912
Epoch 7, train accuracy: 100.0000%, valid. accuracy: 98.9000%, valid. best loss: 0.031912
Epoch 8, train accuracy: 100.0000%, valid. accuracy: 98.9400%, valid. best loss: 0.031912
Epoch 9, train accuracy: 100.0000%, valid. accuracy: 99.0800%, valid. best loss: 0.031912
Epoch 10, train accuracy: 100.0000%, valid. accuracy: 99.1200%, valid. best loss: 0.031912
Epoch 11,